In [ ]:
import os, sys
from typing import Dict

from pydantic import BaseModel
from dotenv import load_dotenv, dotenv_values
from fastapi import FastAPI

load_dotenv()
if (python_path := dotenv_values().get('PYTHONPATH')) and python_path not in sys.path: sys.path.append(python_path)

from src.apps.sample_chat.tools.web_search import query_web_search

from src.apps.sample_chat.chains.chains import (
    bug_step1_chain,
    bug_step2_chain,
    default_chain,
    enhance_step1_chain,
    parse_intent_chain,
    read_prompt_template,
)
from src.apps.sample_chat.database.database import query_db

from src.apps.sample_chat.memory.memory import (
    get_chat_history,
    load_conversation_history,
    log_bot_message,
    log_user_message,
)

app = FastAPI()

In [ ]:
class UserRequest(BaseModel):
    user_message: str


#CUR_DIR = os.path.dirname(os.path.abspath(__file__))
ROOT_DIR = os.getenv("PYTHONPATH")
INTENT_LIST_TXT = os.path.join(ROOT_DIR, "src/apps/sample_chat/prompts", "intent_list.txt")

In [ ]:
@app.post("/qna/{conversation_id}")
def gernerate_answer(req: UserRequest, conversation_id: str) -> Dict[str, str]:
    history_file = load_conversation_history(conversation_id)

    context = req.model_dump()
    context["input"] = context["user_message"]
    context["intent_list"] = read_prompt_template(INTENT_LIST_TXT)
    context["chat_history"] = get_chat_history(conversation_id)

    # intent = parse_intent_chain(context)["intent"]
    intent = parse_intent_chain.invoke(context)
    print("*"*50)
    print("intent="+str(intent))
    print("*"*50)
    if intent["intent"] == "bug":
        print("1"*50)
        context["related_documents"] = query_db(context["user_message"])

        answer = ""
        for step in [bug_step1_chain, bug_step2_chain]:
            context = step.invoke(context)
            answer +=  context["bug_analysis" if step is bug_step1_chain else "output"]
            answer += "\n\n"
    elif intent["intent"] == "enhancement":
        print("2"*50)
        answer = enhance_step1_chain.invoke(context)['output']
    else:
        print("3"*50)
        print("context="+str(context))
        print("3"*50)
        context["related_documents"] = query_db(context["user_message"])
        context["compressed_web_search_results"] = query_web_search(
            context["user_message"]
        )
        answer = default_chain.invoke(context)['output']

    log_user_message(history_file, req.user_message)
    log_bot_message(history_file, answer)
    return {"answer": answer}

In [ ]:
def run_test(test_conversation_id: str, test_user_message: str):
    """
    테스트를 실행하고 결과를 출력하는 함수
    """
    print(f"--- 테스트 시작 (ID: {test_conversation_id}) ---")
    
    # 1. FastAPI가 만들어주던 입력값들을 직접 생성
    # UserRequest 모델의 인스턴스 생성
    test_request = UserRequest(user_message=test_user_message)

    # 2. gernerate_answer 함수를 일반 함수처럼 호출
    try:
        response = gernerate_answer(req=test_request, conversation_id=test_conversation_id)
        
        # 3. 결과 확인
        print("\n--- 최종 응답 ---")
        print(response)
        
    except Exception as e:
        print(f"\n--- 오류 발생 ---")
        print(f"오류: {e}")
        
    finally:
        # 테스트 후 생성된 파일 정리
        history_file = os.path.join(ROOT_DIR, "src/apps/sample_chat/chat_history", f"{test_conversation_id}.json") 
        if os.path.exists(history_file):
            os.remove(history_file)
            print(f"\n생성된 히스토리 파일 ({history_file})을 삭제했습니다.")
        else: 
            print(history_file)    
        print(f"--- 테스트 종료 (ID: {test_conversation_id}) ---")


# 테스트 실행
test_conversation_id_1 = "manual_test_bug_001"
#test_user_message_1 = "홈페이지 게시판 저장버튼이 안눌려요. 버그같아요."
test_user_message_1 = "홈페이지 메인화면을 enhancement 하고 싶어요."

run_test(test_conversation_id_1, test_user_message_1)

print("\n" + "="*50 + "\n")

#test_conversation_id_2 = "manual_test_bug_001"
#test_user_message_2 = "이 기능 좀 개선해주세요. 제가 뭐가 안눌린다고 했죠?"
#run_test(test_conversation_id_2, test_user_message_2)
